# Introduction to the Mean-Field Approximate Optimization Algorithm

For more details, please consult our paper: https://doi.org/10.1103/PRXQuantum.4.030335

In [ ]:
using QAOA, LinearAlgebra
import Random, Distributions

using PyPlot

In [ ]:
# schedule
p = 100
τ = 0.5
γ = τ .* ((1:p) .- 1/2) ./ p |> collect
β = τ .* (1 .- (1:p) ./ p) |> collect
β[p] = τ / (4 * p)

times = range(0, 1, p+1);

__Defining a random Sherrington-Kirkpatrick instance:__

In [ ]:
N = 5
σ2 = 1.0

Random.seed!(1)
J = rand(Distributions.Normal(0, σ2), N, N) ./ sqrt(N) 
J[diagind(J)] .= 0.0
J = UpperTriangular(J)
J = J + transpose(J)

In [ ]:
mf_problem = Problem(p, J)

__Evolve without saving the history:__

In [ ]:
# initial spins
S = [[1., 0., 0.] for _ in 1:N-1] # fix final spin (i.e. leave it out)

# evolution
evolve!(S, mf_problem.local_fields, mf_problem.couplings, β, γ)

__Evolve while saving the history:__

In [ ]:
# initial spins
S = [[[1., 0., 0.] for _ in 1:N-1] for _ in 1:p+1]

# evolution with history
evolve!(S, mf_problem.local_fields, mf_problem.couplings, β, γ);

In [ ]:
# helper function to reformat the data
get_spin_data = n -> mapreduce(permutedims, vcat, [S[k][n] for k in 1:p+1]) |> transpose;

In [ ]:
# plot x, y, and z of all spins 
figure(figsize=((N - 1) * 2.2, 2))

for n in 1:N - 1
    subplot(1, N - 1, n)
    plot(times, get_spin_data(n)[1, 1:end])
    plot(times, get_spin_data(n)[2, 1:end])
    plot(times, get_spin_data(n)[3, 1:end])
    xlim(0, 1)
    xlabel("t/T")
    ylabel("n_" * string(n))
end
tight_layout()

__Energy expectation value:__

In [ ]:
expectation(S[end], mf_problem.local_fields, mf_problem.couplings)

__Solution vector in mean-field approximation:__

In [ ]:
mean_field_solution(S[end])

## Using the ODE solver

In [ ]:
T_final = p*τ
schedule_function = t -> t/T_final
sol = QAOA.evolve(mf_problem.local_fields, mf_problem.couplings, T_final, schedule_function)
sol.u[end]

### Using the tensor problem definition

Note that `TensorProblem` can also deal with arbitrary higher-order tensors.

In [ ]:
    # create tensors
    xtensor = Dict([(i, ) => 1.0 for i in 1:mf_problem.num_qubits])
    ztensor = Dict()
    for (i, h_i) in enumerate(mf_problem.local_fields)
        if h_i != 0.0
            ztensor[(i,)] = h_i
        end
    end
    
    for i in 1:mf_problem.num_qubits
        for j in i+1:mf_problem.num_qubits
            if mf_problem.couplings[i, j] != 0.0
                ztensor[(i, j)] = mf_problem.couplings[i, j]
            end
        end
    end 

In [ ]:
# define tensor problem
tensor_problem = TensorProblem(mf_problem.num_qubits, xtensor, ztensor)

In [ ]:
sol = QAOA.evolve(tensor_problem, T_final, t -> 1 - schedule_function(t), schedule_function)
sol.u[end]